In [ ]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from util.data_generate import *
from util.methods import *
import csv
import random
from util.calculate import MultivariateNormalMinimum
#best
p = 200
#betas = ['smallest','cluster','dynamic','cyclic','independent']
#betas = ['smallest','cluster','dynamic','cyclic']
betas = ['independent']
seed, J, n_iter, num_repeats = 1, 10000, 150, 30
np.random.seed(seed=seed)
Sigma1 = np.loadtxt('Data_generate/Sigma1.txt')
Sigma1 = Sigma1 * 4

Sigma2 = np.loadtxt('Data_generate/group_cov.txt')
std2 = np.sqrt(np.diag(Sigma2))
R2 = Sigma2 / np.outer(std2, std2)
mu = np.loadtxt('Data_generate/mu.txt')

Sigma0 = np.diag(np.diag(Sigma2))

for m in [10,]:
    U3, ids3 = cyclic_multstart(p, m, mu, std2, R2, Sigma2, J=J, mult=50)
    z_samples = np.random.randn(J*100, m)
    G_real = np.mean(np.min(mu[ids3] + (U3 @ z_samples.T).T, axis=1))
    print(ids3,end=": ")
    print(G_real)
#--------------------------------------------------------------------
    G_real = 46.8348546986495
#--------------------------------------------------------------------
    cluster_labels = cluster_subjects(R2, n_clusters=m, method='average')
    for beta in betas:

        detail_csv_path = f'{p}_{m}_{beta}_details.csv'
        detail_records = []  # 用于批量存储

        csv_path = f'{p}_{m}_{beta}.csv'
        csv_path1 = f'{p}_{m}_{beta}_pre.csv'

        # Initialize CSV
        with open(csv_path, 'w') as f:
            f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter+1)]) + '\n')
        with open(csv_path1, 'w') as f:
            f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter+1)]) + '\n')

        for repeat in tqdm(range(num_repeats)):
            # Initial setup
            seed = seed + repeat
            
            # Initialize variables
            mu_0 = np.full(p, 70)
            mu_hat, var_hat, obs_num = np.zeros(p), np.ones(p), np.zeros(p)
            
            # Initial observation
            N_0 = math.ceil(p / m)
            for i in range(N_0):
                ids = efficient_initialization(p, m, obs_num, mu_0, std2, R2, Sigma2, J)
                obs_num[ids] += 1
                indices = ids.copy()
                obs = np.random.multivariate_normal(mu[ids], Sigma2[np.ix_(ids, ids)])
                X_n = ids.copy()
                
                # Update estimates
                Gamma = Sigma2[np.ix_(ids, ids)]
                inv_mat = np.linalg.inv(Sigma1[np.ix_(ids, ids)] + Gamma)
                delta = (obs - mu_0[ids]).reshape(-1, 1)
                
                COV_x = Sigma1[:, ids]
                adjustments = (COV_x @ inv_mat @ delta).flatten()
                mu_hat = mu_0 + adjustments
                adjustments = np.diag(COV_x @ inv_mat @ COV_x.T)
                var_hat = np.diag(Sigma1) - adjustments
                
            # Main loop
            G = [] #用于计算累积遗憾
            G_pre = [] 
            for t in range(N_0+1, n_iter+1):
                dynamic_beta = np.sqrt(6 * np.log(t) / np.maximum(obs_num, 1))
                if beta == 'dynamic':
                    U, ids = improvement(p, m, mu_hat - dynamic_beta * np.sqrt(var_hat), std2, R2, Sigma2, J=J)
                elif beta == 'cluster': 
                    ids = find_min_mu_in_clusters(mu_hat - dynamic_beta * np.sqrt(var_hat), cluster_labels)
                    sub_Sigma_cluster = Sigma2[np.ix_(ids, ids)]
                    U = np.linalg.cholesky(sub_Sigma_cluster) 
                elif beta == 'cyclic': 
                    U, ids = cyclic_multstart(p,m,mu_hat - dynamic_beta * np.sqrt(var_hat),std2, R2,Sigma2,J=100000,mult=10)
                elif beta == 'independent': 
                    ids = cyclic_multstart(p,m,mu_hat - dynamic_beta * np.sqrt(var_hat),std2, R2,Sigma0,J=100000,mult=10)[1]
                    sub_Sigma_indep = Sigma2[np.ix_(ids, ids)]
                    U = np.linalg.cholesky(sub_Sigma_indep)
                else:
                    ids = np.argsort(mu_hat - dynamic_beta * np.sqrt(var_hat))[:m]
                    sub_Sigma_smallest = Sigma2[np.ix_(ids, ids)]
                    U = np.linalg.cholesky(sub_Sigma_smallest) 
                obs_num[ids] += 1
                indices = np.vstack((indices, ids))
                
                newobs = np.random.multivariate_normal(mu[ids], Sigma2[np.ix_(ids, ids)])
                X_n, obs = update_subject_stats(obs_num, X_n, obs, ids, newobs)
                
                # Update Gamma matrix
                len_Xn = len(X_n)
                Gamma = np.zeros((len_Xn, len_Xn))
                for i in range(len_Xn):
                    for j in range(i, len_Xn):
                        count = np.sum((indices == X_n[i]).any(1) & (indices == X_n[j]).any(1))
                        Gamma[i,j] = Sigma2[X_n[i],X_n[j]] * count / (obs_num[X_n[i]] * obs_num[X_n[j]])
                Gamma += Gamma.T
                np.fill_diagonal(Gamma, np.diag(Gamma)/2)
                
                # Update estimates
                COV_XX = Sigma1[np.ix_(X_n, X_n)]
                inv_mat = np.linalg.inv(COV_XX + Gamma)
                delta = (obs - mu_0[X_n]).reshape(-1, 1)
                
                COV_x = Sigma1[:, X_n]
                adjustments = (COV_x @ inv_mat @ delta).flatten()
                mu_hat = mu_0 + adjustments
                adjustments = np.diag(COV_x @ inv_mat @ COV_x.T)
                var_hat = np.diag(Sigma1) - adjustments

                # Calculate G
                z_samples = np.random.randn(J*10, m)
                observe_value = np.mean(np.min(mu[ids] + (U @ z_samples.T).T, axis=1))
                G.append(observe_value)

                modules_str = '_'.join(map(str, ids))
                detail_records.append([beta, repeat+1, t, modules_str, f'{observe_value:.6f}'])

                Gmin = 1000
                idmin = []
                z_samples2 = np.random.randn(J, m)
                for id in indices:
                    sub_Sigma = Sigma2[np.ix_(id, id)]
                    B = np.linalg.cholesky(sub_Sigma)
                    Gtemp = np.mean(np.min(mu_hat[id] + (B @ z_samples2.T).T, axis=1))
                    if Gtemp < Gmin:
                        idmin = id.copy()
                        Gmin = Gtemp
                ids1 = idmin

                sub_Sigma = Sigma2[np.ix_(ids1, ids1)]
                U1 = np.linalg.cholesky(sub_Sigma)
                G_pre.append(np.mean(np.min(mu[ids1] + (U1 @ z_samples.T).T, axis=1)))
            
            # Save results
            with open(csv_path, 'a') as f:
                f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G)-G_real]) + '\n')
            with open(csv_path1, 'a') as f:
                f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G_pre)-G_real]) + '\n')

        with open(detail_csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Algorithm', 'Repetition', 'Iteration', 'Selected_Modules', 'Expected_Minimum'])
            writer.writerows(detail_records)

  0%|          | 0/30 [00:00<?, ?it/s]

[125, 91, 87, 141, 13, 104, 9, 7, 10, 8]: 46.8348546986495


100%|██████████| 30/30 [1:53:37<00:00, 227.24s/it]


In [2]:
!shutdown -h +1 

Shutdown scheduled for Tue 2025-11-18 12:58:02 CST, use 'shutdown -c' to cancel.


* worst: [31, 34, 29, 40, 24, 36, 43, 35]
* random: [1, 20, 47, 35, 7, 8, 17, 37]
* best: [10, 11, 48, 47, 9, 45, 43, 34]

In [ ]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from util.data_generate import *
from util.methods import *
import csv
#best
p = 50
m=16
betas = ['smallest','cluster','dynamic']
seed, J, n_iter, num_repeats = 1, 10000, 150, 30
np.random.seed(seed=seed)
Sigma1 = np.loadtxt('Data_generate/Sigma1.txt')
Sigma1 = Sigma1 * 4

Sigma2 = np.loadtxt('Data_generate/group_cov.txt')
std2 = np.sqrt(np.diag(Sigma2))
R2 = Sigma2 / np.outer(std2, std2)
mu = np.loadtxt('Data_generate/mu.txt')

#U, ids1 = improvement(p, m, mu, std2, R2, Sigma2, J=J)
ids1 = [11, 34, 1, 35, 9, 28, 0, 7, 31, 29, 3, 2, 4, 33, 32, 30]
ids2 = [4,11,29,28,12,7,32,0,31,35,30,3,34,2,1,33]

In [ ]:
A = cyclic_multstart(p, m, mu, std2, R2, Sigma2, J=J, mult = 100)
ids2 = A[1]

In [ ]:
z_samples2 = np.random.randn(1000000, m)
ids1 = ids1
sub_Sigma = Sigma2[np.ix_(ids1, ids1)]
U1 = np.linalg.cholesky(sub_Sigma)
e1 = (np.mean(np.min(mu[ids1] + (U1 @ z_samples2.T).T, axis=1)))
e1

In [ ]:
ids2 = ids2
sub_Sigma = Sigma2[np.ix_(ids2, ids2)]
U2 = np.linalg.cholesky(sub_Sigma)
e2 = (np.mean(np.min(mu[ids2] + (U2 @ z_samples2.T).T, axis=1)))
e2

In [ ]:
from util.calculate import MultivariateNormalMinimum
# 初始化
calculator = MultivariateNormalMinimum(mu[ids1], Sigma2[np.ix_(ids1, ids1)])

# 简单计算
for i in range(30):
    result_exact = calculator.compute_R(max_dim=21, n_samples=100000,seed = i)
    print(result_exact)

In [ ]:
mean_val, lower, upper 

In [ ]:
99/99.2